<a href="https://colab.research.google.com/github/PaiwandAlAhmady/YOLOv11_Object-Segmentation/blob/main/YOLOv8_seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv8 Notebook for RoboCup@Work

## Dependencies

In [ ]:
%pip install -q ultralytics -U
%pip install -q depthai==3.0.0rc2 -U
%pip install -q depthai-nodes==0.3.0 -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.3/224.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Lädt Trainingsdatensatz von Drive auf lokalen Colab Speicher -> effizienter
!cp -r /content/drive/MyDrive/RoboCup_Trainingsdatensatz /content/

^C


In [ ]:
from ultralytics import YOLO

model_yolov8_n = YOLO("yolov8n-seg.pt")
model_yolov8_s = YOLO("yolov8s-seg.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Train Model

In [ ]:
# --- Repro & Umgebungs-Setup (vor Torch/Ultralytics importieren!) ---
import os
os.environ["PYTHONHASHSEED"] = "0"
# Für deterministische GEMMs (CUDA 11+). Falls Probleme: Zeile auskommentieren.
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"

import random, numpy as np, torch
torch.use_deterministic_algorithms(True, warn_only=True)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(0); np.random.seed(0)
torch.manual_seed(0); torch.cuda.manual_seed_all(0)

# --- Imports ---
from ultralytics import YOLO
from datetime import datetime
from google.colab import runtime

# --- Pfade & gemeinsame Trainingsargumente ---
data_yaml = "/content/drive/MyDrive/RoboCup_Trainingsdatensatz/data.yaml"
common = dict(
    data=data_yaml,
    imgsz=832, batch=32, workers=4,
    epochs=300, patience=60,
    cache=True, deterministic=True, seed=0,
    save=True, save_period=25,
    # Augmentations (an YOLOv11 angelehnt):
    fliplr=0.50, flipud=0.10, degrees=10.0, shear=3.0, perspective=0.000,
    scale=0.50, translate=0.20,
    hsv_h=0.02, hsv_s=0.80, hsv_v=0.50,
    mosaic=0.80, close_mosaic=10, mixup=0.10, copy_paste=0.20,
    erasing=0.50
)

run_n = f"y8nseg_832_b32_pat60_{datetime.now().strftime('%Y%m%d_%H%M')}"
model_n = YOLO("yolov8n-seg.pt")  # alternativ: "yolov8n-seg.yaml" für From-Scratch
model_n.train(project="/content/drive/MyDrive/YOLO_Runs/segment", name=run_n, **common)

# --- Training: v8s-seg ---
run_s = f"y8sseg_832_b32_pat60_{datetime.now().strftime('%Y%m%d_%H%M')}"
model_s = YOLO("yolov8s-seg.pt")  # alternativ: "yolov8s-seg.yaml" für From-Scratch
model_s.train(project="/content/drive/MyDrive/YOLO_Runs/segment", name=run_s, **common)

runtime.unassign()


Ultralytics 8.3.208 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.2, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/RoboCup_Trainingsdatensatz/data.yaml, degrees=10.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.5, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.8, hsv_v=0.5, imgsz=832, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=0.8, multi_scale=False, name=y8nseg_832_b32_pat60_20251009_2123, nbs=64, nms=False, opset=None, optimize=False, optimiz

## Convertion for further purposes

In [ ]:
from ultralytics import YOLO
m = YOLO("/content/drive/MyDrive/YOLO_Runs/segment/y8nseg_832_b32_pat60_20251009_2123/weights/best.pt")
m.export(format="onnx", imgsz=832)  # nimm deine Trainings-Size (z.B. 832 oder 640)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.208 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLOv8n-seg summary (fused): 85 layers, 3,258,649 parameters, 0 gradients, 11.3 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/YOLO_Runs/segment/y8nseg_832_b32_pat60_20251009_2123/weights/best.pt' with input shape (1, 3, 832, 832) BCHW and output shape(s) ((1, 39, 14196), (1, 32, 208, 208)) (6.5 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim>=0.1.67', 'onnxruntime'] not found, attempting AutoUpdate...

requirements: AutoUpdate

'/content/drive/MyDrive/YOLO_Runs/segment/y8nseg_832_b32_pat60_20251009_2123/weights/best.onnx'